In [2]:

# Librerías de manejo de datos 
import pandas as pd
from pandas import concat
import numpy as np

# Librerías de machine learning
import sklearn
from sklearn import preprocessing
from sklearn import metrics as sk
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

# Librerías de visualización
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout
import plotly.tools as tls
tls.set_credentials_file(username='raulMrello', api_key='qX9S30YRQ866mGF9b89u')

# TA-Lib: 
import talib

# Otras utilidades
import datetime
import time
import os
import sys

print('Done!')

Done!


In [6]:
#3. Carga el archivo en un dataframe
import pandas as pd
eurusd = pd.read_csv('EURUSD_M1.csv', sep=",") 
eurusd.head()


,<TICKER>,<DTYYYYMMDD>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4
2,EURUSD,20010102,230300,0.9505,0.9507,0.9505,0.9506,4
3,EURUSD,20010102,230400,0.9506,0.9506,0.9506,0.9506,4
4,EURUSD,20010102,230500,0.9506,0.9506,0.9506,0.9506,4


In [8]:
# elimina la columna ticker y renombra el resto de columnas
eurusd.columns = ['TICKER','DATE','TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOL']
eurusd.head()

,TICKER,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,EURUSD,20010102,230100,0.9507,0.9507,0.9507,0.9507,4
1,EURUSD,20010102,230200,0.9506,0.9506,0.9505,0.9505,4
2,EURUSD,20010102,230300,0.9505,0.9507,0.9505,0.9506,4
3,EURUSD,20010102,230400,0.9506,0.9506,0.9506,0.9506,4
4,EURUSD,20010102,230500,0.9506,0.9506,0.9506,0.9506,4


In [10]:
eurusd = eurusd.drop(columns=['TICKER'])
eurusd.head()

,DATE,TIME,OPEN,HIGH,LOW,CLOSE,VOL
0,20010102,230100,0.9507,0.9507,0.9507,0.9507,4
1,20010102,230200,0.9506,0.9506,0.9505,0.9505,4
2,20010102,230300,0.9505,0.9507,0.9505,0.9506,4
3,20010102,230400,0.9506,0.9506,0.9506,0.9506,4
4,20010102,230500,0.9506,0.9506,0.9506,0.9506,4


In [25]:
# crea una nueva columna de timestamp uniendo date & time
df = eurusd.copy()
df['DATE'] = df['DATE'].astype(str)
df['TIME'] = df['TIME'].astype(str)
df.DATE = df.DATE.apply(lambda x: x.zfill(8))
df.TIME = df.TIME.apply(lambda x: x.zfill(6))
df['timestamp'] = df.DATE + ' ' + df.TIME
print(df.head())



       DATE    TIME    OPEN    HIGH     LOW   CLOSE  VOL        timestamp
0  20010102  230100  0.9507  0.9507  0.9507  0.9507    4  20010102 230100
1  20010102  230200  0.9506  0.9506  0.9505  0.9505    4  20010102 230200
2  20010102  230300  0.9505  0.9507  0.9505  0.9506    4  20010102 230300
3  20010102  230400  0.9506  0.9506  0.9506  0.9506    4  20010102 230400
4  20010102  230500  0.9506  0.9506  0.9506  0.9506    4  20010102 230500


In [26]:
# elimino las columnas que no voy a utilizar
df = df.drop(columns=['DATE', 'TIME', 'VOL'])
df.head()

,OPEN,HIGH,LOW,CLOSE,timestamp
0,0.9507,0.9507,0.9507,0.9507,20010102 230100
1,0.9506,0.9506,0.9505,0.9505,20010102 230200
2,0.9505,0.9507,0.9505,0.9506,20010102 230300
3,0.9506,0.9506,0.9506,0.9506,20010102 230400
4,0.9506,0.9506,0.9506,0.9506,20010102 230500


In [27]:
# cambio el formato del timestamp a formato horario
df['timestamp'] = df['timestamp'].map(lambda x: datetime.datetime.strptime(x, '%Y%m%d %H%M%S'))  
df.head()

,OPEN,HIGH,LOW,CLOSE,timestamp
0,0.9507,0.9507,0.9507,0.9507,2001-01-02 23:01:00
1,0.9506,0.9506,0.9505,0.9505,2001-01-02 23:02:00
2,0.9505,0.9507,0.9505,0.9506,2001-01-02 23:03:00
3,0.9506,0.9506,0.9506,0.9506,2001-01-02 23:04:00
4,0.9506,0.9506,0.9506,0.9506,2001-01-02 23:05:00


In [28]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
print(df.head())

     OPEN    HIGH     LOW   CLOSE           timestamp
0  0.9507  0.9507  0.9507  0.9507 2001-01-02 23:01:00
1  0.9506  0.9506  0.9505  0.9505 2001-01-02 23:02:00
2  0.9505  0.9507  0.9505  0.9506 2001-01-02 23:03:00
3  0.9506  0.9506  0.9506  0.9506 2001-01-02 23:04:00
4  0.9506  0.9506  0.9506  0.9506 2001-01-02 23:05:00


In [30]:
# establezco el timestamp como índice
df.set_index('timestamp', inplace = True)
print(df.head())

                       OPEN    HIGH     LOW   CLOSE
timestamp                                          
2001-01-02 23:01:00  0.9507  0.9507  0.9507  0.9507
2001-01-02 23:02:00  0.9506  0.9506  0.9505  0.9505
2001-01-02 23:03:00  0.9505  0.9507  0.9505  0.9506
2001-01-02 23:04:00  0.9506  0.9506  0.9506  0.9506
2001-01-02 23:05:00  0.9506  0.9506  0.9506  0.9506


In [31]:
# reajusto el índice para que tenga una cadencia de 1min
df = df.reindex(pd.date_range(min(df.index), max(df.index), freq="1min"))
df.head()

,OPEN,HIGH,LOW,CLOSE
2001-01-02 23:01:00,0.9507,0.9507,0.9507,0.9507
2001-01-02 23:02:00,0.9506,0.9506,0.9505,0.9505
2001-01-02 23:03:00,0.9505,0.9507,0.9505,0.9506
2001-01-02 23:04:00,0.9506,0.9506,0.9506,0.9506
2001-01-02 23:05:00,0.9506,0.9506,0.9506,0.9506


In [37]:
# chequeo la cantidad de datos NaN en la tabla
df.isnull().sum()


OPEN     3092181
HIGH     3092181
LOW      3092181
CLOSE    3092181
dtype: int64

In [38]:
# visualiza el candlestick (sólo 2000 muestras)
dfview = df[0:2000]
trace = go.Candlestick(x=dfview.index,
                       open=dfview.OPEN,
                       high=dfview.HIGH,
                       low=dfview.LOW,
                       close=dfview.CLOSE)
data = [trace]
py.iplot(data)

In [ ]:
# relleno los espacios en blanco con el último valor válido encontrado
df.fillna(method='ffill', inplace =True)
dfview = df
trace = go.Candlestick(x=dfview.index,
                       open=dfview.OPEN,
                       high=dfview.HIGH,
                       low=dfview.LOW,
                       close=dfview.CLOSE)
data = [trace]
py.iplot(data)